In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import spacy
from spacy.lang.pt import Portuguese
from nltk.corpus import stopwords
import time

import warnings
warnings.filterwarnings('ignore')

In [2]:
nlp = spacy.load('pt_core_news_lg')
parser = Portuguese()

# Preparação dos dados

In [3]:
# Import data
df_traindata = pd.read_csv('../data/traindata.csv', delimiter=';')
df_testdata = pd.read_csv('../data/testdata.csv', delimiter=';')

# Data preparation for Binary
df_traindata['category'] = df_traindata.reset_index().category.map({'covid':0, 'seloturismo':1, 'tuberculose':2})
df_testdata['category'] = df_testdata.reset_index().category.map({'covid':0, 'seloturismo':1, 'tuberculose':2})

In [4]:
# monta dataset
df_total = pd.concat([df_traindata, df_testdata])

textos = df_total['input'].tolist()
rotulos = df_total['category'].tolist()

# Normalizing and TF-IDF preparation
stop_words = stopwords.words('portuguese')

# Transforma os textos em vetores de recursos usando TF-IDF
vectorizer = TfidfVectorizer(lowercase=True, stop_words=stop_words, strip_accents='unicode', token_pattern="[A-Za-z]+")
X = vectorizer.fit_transform(textos)

# Divide o dataset em treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, rotulos, test_size=0.2)

# Treinamento do modelo com os dados originais

In [5]:
# Cria o classificador KDTree
start_time_train = time.time()
kdtree = NearestNeighbors(n_neighbors=1, n_jobs=-1, algorithm='auto')
kdtree.fit(X_train)
end_time_train = time.time()
print("Tempo de treino: {:.5f} segundos".format(end_time_train - start_time_train))

# Realize a inferência usando o classificador
tempo_ini = time.time()
_, indices = kdtree.kneighbors(X_test)
y_pred = [y_train[i[0]] for i in indices]
tempo_fim = time.time()

print("Tempo médio de inferência: {:.5f} segundos".format((tempo_fim - tempo_ini) / X_test.shape[0]))
print(classification_report(y_test, y_pred, target_names=['0', '1', '2']))


Tempo de treino: 0.00000 segundos
Tempo médio de inferência: 0.00005 segundos
              precision    recall  f1-score   support

           0       0.79      0.35      0.48        75
           1       0.67      0.99      0.80       169
           2       0.89      0.52      0.66        92

    accuracy                           0.72       336
   macro avg       0.78      0.62      0.65       336
weighted avg       0.76      0.72      0.69       336



# Inferência para 3.721 registros

In [6]:
df_novotestdata = pd.read_csv('../data/novodatasetgerado.csv', delimiter=';')
df_novotestdata['category'] = df_novotestdata.reset_index().category.map({'covid':0, 'seloturismo':1, 'tuberculose':2})

rotulos = df_novotestdata['category'].tolist()

novos_dados_vetorizados = vectorizer.transform(df_novotestdata['input'].tolist())

# Realize a inferência usando o classificador
tempo_ini = time.time()
_, indices = kdtree.kneighbors(novos_dados_vetorizados)
y_pred = [y_train[i[0]] for i in indices]
tempo_fim = time.time()

print('######### ', novos_dados_vetorizados.shape[0], 'registros.')
print("Tempo médio de inferência: {:.5f} segundos".format((tempo_fim - tempo_ini) / novos_dados_vetorizados.shape[0]))
print(classification_report(rotulos, y_pred, target_names=['0', '1', '2']))

#########  3721 registros.
Tempo médio de inferência: 0.00002 segundos
              precision    recall  f1-score   support

           0       0.55      0.15      0.24      1158
           1       0.40      0.94      0.56      1236
           2       0.70      0.26      0.38      1327

    accuracy                           0.45      3721
   macro avg       0.55      0.45      0.39      3721
weighted avg       0.56      0.45      0.40      3721



# Inferência para MI registros

In [7]:
headers = ['input', 'category']
df_novotestdata = pd.read_csv('../data/novodatasetgerado-frasesduplicadas.csv', delimiter=';', error_bad_lines=False, names=headers, header=0)
df_novotestdata['category'] = df_novotestdata.reset_index().category.map({'covid':0, 'seloturismo':1, 'tuberculose':2})


In [8]:
subset = df_novotestdata.head(1000000)

rotulos = subset['category'].tolist()

novos_dados_vetorizados = vectorizer.transform(subset['input'].tolist())

# Realize a inferência usando o classificador
tempo_ini = time.time()
_, indices = kdtree.kneighbors(novos_dados_vetorizados)
y_pred = [y_train[i[0]] for i in indices]
tempo_fim = time.time()

print('######### ', novos_dados_vetorizados.shape[0], 'registros.')
print("Tempo médio de inferência: {:.5f} segundos".format((tempo_fim - tempo_ini) / novos_dados_vetorizados.shape[0]))
print(classification_report(rotulos, y_pred, target_names=['0', '1', '2']))

#########  1000000 registros.
Tempo médio de inferência: 0.00002 segundos
              precision    recall  f1-score   support

           0       0.55      0.15      0.24    311116
           1       0.40      0.94      0.56    332104
           2       0.70      0.26      0.38    356780

    accuracy                           0.45   1000000
   macro avg       0.55      0.45      0.39   1000000
weighted avg       0.56      0.45      0.40   1000000

